In [1]:
import pandas as pd  


In [2]:
# Read all available data files, raise a clear error if a file is not found
import os
from pathlib import Path

# Get project root directory (parent of notebooks directory)
# This works whether running from project root or notebooks directory
if Path('config.yaml').exists():
    project_root = Path('.').resolve()
elif Path('../config.yaml').exists():
    project_root = Path('..').resolve()
else:
    # Try to find it by going up from current directory
    current = Path.cwd()
    while current != current.parent:
        if (current / 'config.yaml').exists():
            project_root = current.resolve()
            break
        current = current.parent
    else:
        project_root = Path('.').resolve()

# Change to project root for consistent paths
os.chdir(project_root)

# Set data directory (use absolute path)
data_dir = project_root / 'data' / 'expression'
dataset = 'GSE28042'

print(f"Project root: {project_root}")
print(f"Data directory: {data_dir}")
print(f"Dataset: {dataset}")
print("=" * 60)

# Define file paths
files = {
    'probe_to_gene': data_dir / f'{dataset}_probe_to_gene.csv',
    'gene_expression': data_dir / f'{dataset}_gene_expression.csv',
    'combined': data_dir / f'{dataset}_gene_expression_with_phenotypes.csv'
}

# Load dataframes
dataframes = {}
for name, filepath in files.items():
    if filepath.exists():
        print(f"Loading {name}: {filepath.name}")
        if name == 'gene_expression':
            # Set gene_symbol as index for gene expression matrix
            df = pd.read_csv(filepath, index_col=0)
        else:
            df = pd.read_csv(filepath)
        dataframes[name] = df
        print(f"  OK Loaded: {df.shape[0]} rows × {df.shape[1]} columns")
    else:
        raise FileNotFoundError(f"File not found: {filepath}")

print("=" * 60)

# Assign to variables for easy access
probe_to_gene = dataframes['probe_to_gene']
gene_expression = dataframes['gene_expression']
combined = dataframes['combined']

print(f"\nData loaded successfully!")
print(f"  - Probe-to-gene mapping: {probe_to_gene.shape}")
print(f"  - Gene expression matrix: {gene_expression.shape} (genes × samples)")
print(f"  - Combined dataset: {combined.shape} (samples × genes + phenotypes)")


Project root: C:\Users\User\Desktop\Personal\github\projects\gwas\gwas-tutorial
Data directory: C:\Users\User\Desktop\Personal\github\projects\gwas\gwas-tutorial\data\expression
Dataset: GSE28042
Loading probe_to_gene: GSE28042_probe_to_gene.csv
  OK Loaded: 30936 rows × 4 columns
Loading gene_expression: GSE28042_gene_expression.csv
  OK Loaded: 18614 rows × 94 columns
Loading combined: GSE28042_gene_expression_with_phenotypes.csv
  OK Loaded: 94 rows × 18647 columns

Data loaded successfully!
  - Probe-to-gene mapping: (30936, 4)
  - Gene expression matrix: (18614, 94) (genes × samples)
  - Combined dataset: (94, 18647) (samples × genes + phenotypes)


In [3]:
# Define function to parse Characteristics Ch1 column
# This will be used for extracting all phenotype variables

def parse_characteristics(characteristics_str):
    """
    Parse the Characteristics Ch1 string to extract key-value pairs.
    
    Format: "key1: value1; key2: value2; ..."
    """
    if pd.isna(characteristics_str) or characteristics_str == '':
        return {}
    
    result = {}
    # Split by semicolon
    pairs = str(characteristics_str).split(';')
    for pair in pairs:
        pair = pair.strip()
        if ':' in pair:
            key, value = pair.split(':', 1)
            key = key.strip().lower()
            value = value.strip()
            result[key] = value
    return result

print("Function parse_characteristics() defined.")
print("This will be used to extract all phenotype variables in the next cell.")


Function parse_characteristics() defined.
This will be used to extract all phenotype variables in the next cell.


In [4]:
# Extract ALL phenotype variables from Characteristics Ch1
# This will create separate columns for each phenotype variable found
# Then create binary disease_status (0 or 1) and remove Characteristics Ch1 column

print("Extracting all phenotype variables from Characteristics Ch1...")
print("=" * 60)

# Collect all unique keys across all samples
all_phenotype_keys = set()
for idx, row in combined.iterrows():
    char_dict = parse_characteristics(row.get('Characteristics Ch1', ''))
    all_phenotype_keys.update(char_dict.keys())

print(f"Found {len(all_phenotype_keys)} unique phenotype variables:")
for key in sorted(all_phenotype_keys):
    print(f"  - {key}")

print("\n" + "=" * 60)

# Extract each phenotype variable into separate columns
phenotype_data = {}
for key in all_phenotype_keys:
    phenotype_data[key] = []

for idx, row in combined.iterrows():
    char_dict = parse_characteristics(row.get('Characteristics Ch1', ''))
    for key in all_phenotype_keys:
        value = char_dict.get(key, None)
        phenotype_data[key].append(value)

# Add phenotype columns to combined dataframe
for key, values in phenotype_data.items():
    # Create a clean column name (replace spaces with underscores, make lowercase)
    col_name = key.replace(' ', '_').lower()
    combined[col_name] = values
    print(f"Added column: {col_name}")

print("\n" + "=" * 60)

# Now create binary disease_status column (0 or 1) from the extracted text values
print("Creating binary disease_status column (0 or 1)...")
disease_status_binary = []

for idx, row in combined.iterrows():
    # Check if disease_status column exists (from extracted phenotype data)
    disease_status_val = None
    
    # First, check the extracted disease_status column if it exists
    if 'disease_status' in combined.columns:
        status_text = str(row['disease_status']).lower() if pd.notna(row['disease_status']) else ''
        if any(term in status_text for term in ['ipf', 'idiopathic pulmonary fibrosis', 'disease', 'case']):
            disease_status_val = 1
        elif any(term in status_text for term in ['control', 'healthy', 'normal']):
            disease_status_val = 0
    
    # If not found, check disease state column
    if disease_status_val is None and 'disease_state' in combined.columns:
        status_text = str(row['disease_state']).lower() if pd.notna(row['disease_state']) else ''
        if any(term in status_text for term in ['ipf', 'idiopathic pulmonary fibrosis', 'disease', 'case']):
            disease_status_val = 1
        elif any(term in status_text for term in ['control', 'healthy', 'normal']):
            disease_status_val = 0
    
    # If still not found, check Description or Title columns
    if disease_status_val is None:
        desc = str(row.get('Description', '')).lower()
        title = str(row.get('Title', '')).lower()
        if any(term in desc or term in title for term in ['ipf', 'disease', 'case']):
            disease_status_val = 1
        elif any(term in desc or term in title for term in ['control', 'healthy', 'normal', 'n_']):
            disease_status_val = 0
    
    disease_status_binary.append(disease_status_val)

# Replace disease_status column with binary values (0 or 1)
combined['disease_status'] = pd.Series(disease_status_binary, dtype='Int64')  # Int64 allows NaN

# Remove the original Characteristics Ch1 column now that we've extracted everything
if 'Characteristics Ch1' in combined.columns:
    combined = combined.drop(columns=['Characteristics Ch1'])
    print("Removed 'Characteristics Ch1' column (phenotype variables extracted)")

print("\n" + "=" * 60)
print("Phenotype extraction complete!")
print("=" * 60)

# Show summary of phenotype data
print("\nPhenotype Summary:")
print("=" * 60)
phenotype_cols = [col for col in combined.columns if col in [k.replace(' ', '_').lower() for k in all_phenotype_keys]]
for col in phenotype_cols:
    if col in combined.columns:
        print(f"\n{col}:")
        print(f"  Non-null values: {combined[col].notna().sum()}")
        if combined[col].notna().sum() > 0:
            unique_vals = combined[col].dropna().unique()
            if len(unique_vals) <= 10:
                print(f"  Unique values: {list(unique_vals)}")
            else:
                print(f"  Unique values: {len(unique_vals)} (showing first 10: {list(unique_vals[:10])})")

# Show disease status summary (should be 0 or 1)
print(f"\nDisease status (binary - 0 or 1):")
print(f"  Total samples: {len(combined)}")
print(f"  Disease (1): {sum(combined['disease_status'] == 1)}")
print(f"  Control (0): {sum(combined['disease_status'] == 0)}")
print(f"  Missing/Unknown: {sum(combined['disease_status'].isna())}")

# Show sample of the data
print("\n" + "=" * 60)
print("Sample phenotype data:")
print("=" * 60)
display_cols = ['sample_id', 'disease_status'] + [col for col in phenotype_cols if col != 'disease_status' and col in combined.columns]
print(combined[display_cols].head(10))


Extracting all phenotype variables from Characteristics Ch1...
Found 10 unique phenotype variables:
  - age
  - cell type
  - disease status
  - gender
  - outcome
  - survival status
  - time to outcome
  - time to outcomel
  - time to survival
  - years to outcome

Added column: time_to_outcomel
Added column: age
Added column: time_to_outcome
Added column: years_to_outcome
Added column: time_to_survival
Added column: outcome
Added column: cell_type
Added column: disease_status
Added column: gender
Added column: survival_status

Creating binary disease_status column (0 or 1)...
Removed 'Characteristics Ch1' column (phenotype variables extracted)

Phenotype extraction complete!

Phenotype Summary:

time_to_outcomel:
  Non-null values: 2
  Unique values: ['0.594 years', '0.120 years']

age:
  Non-null values: 94
  Unique values: 40 (showing first 10: ['75', '74', '78', '62', '53', '59', '60', '79', '77', '69'])

time_to_outcome:
  Non-null values: 72
  Unique values: 70 (showing first 1

In [7]:
print(combined.head(10))
# df = combined   
combined.to_csv('data/expression/GSE28042_aggregated.csv', index=False)

   sample_id      A1BG  A1BG-AS1      A1CF      A2LD1       A2M     A2ML1  \
0  GSM693801  6.839669  9.365172  3.404173   8.823121  6.323282  8.445686   
1  GSM693754  6.686511  8.687128  4.816308   9.785831  4.668888  9.094248   
2  GSM693789  6.144649  9.306434  2.218785   9.720265  4.836814  8.428688   
3  GSM693823  6.319484  9.469404  2.366918   9.365207  6.225802  8.470793   
4  GSM698454  5.808382  8.813867  3.886297   9.061888  5.341827  8.401290   
5  GSM693755  6.866474  8.943292  4.206750  10.151740  5.241901  8.973537   
6  GSM693821  6.981880  9.416767  5.228652   9.216403  5.188779  7.853235   
7  GSM693791  6.778037  9.204000  3.964618   9.215590  5.874796  8.464651   
8  GSM693806  6.839657  8.785221  2.389324   8.560491  5.831165  7.829425   
9  GSM698440  6.436488  9.360215  5.276815   8.457146  4.509735  8.485283   

     A4GALT     A4GNT       AAAS  ...  time_to_outcomel  age  time_to_outcome  \
0  3.123935  1.272314   9.529857  ...              None   75      2.487

In [ ]:
# analysis

In [11]:
# Load all cleaned data files
import pandas as pd
import os
from pathlib import Path

# Get project root directory (parent of notebooks directory)
# This works whether running from project root or notebooks directory
if Path('config.yaml').exists():
    project_root = Path('.').resolve()
elif Path('../config.yaml').exists():
    project_root = Path('..').resolve()
else:
    # Try to find it by going up from current directory
    current = Path.cwd()
    while current != current.parent:
        if (current / 'config.yaml').exists():
            project_root = current.resolve()
            break
        current = current.parent
    else:
        project_root = Path('.').resolve()

# Change to project root for consistent paths
os.chdir(project_root)

# Get data directory
data_dir = project_root / 'data' / 'expression'

# Find all cleaned data files
# options from config.yaml:
# - GSE93606_cleaned.csv
# - GSE38958_cleaned.csv
# - GSE28042_cleaned.csv
# - GSE33566_cleaned.csv
data = list(data_dir.glob('*GSE93606_cleaned.csv'))
data = pd.read_csv(data[0])    
data.head(10)

,sample_id,A1BG,A1CF,A2LD1,A2M,A2ML1,A4GALT,A4GNT,AAA1,AAAS,...,Contact Address,age_(years),tissue,disease_state,fvc,survival_(months),dlco,composite_end_point,gender,disease_status
0,GSM2458633,2.904471,2.161361,3.765919,6.867662,2.405592,2.881659,1.840449,1.733315,5.745131,...,Dovehouse Street,NaN,whole blood,Idiopathic Pulmonary Fibrosis,NaN,NaN,NaN,NaN,NaN,1
1,GSM2458717,3.339867,2.064593,3.187950,4.717413,2.196021,2.461289,2.198879,1.941956,5.964172,...,Dovehouse Street,NaN,whole blood,Idiopathic Pulmonary Fibrosis,NaN,NaN,NaN,NaN,NaN,1
2,GSM2458574,3.849669,1.973408,3.512561,5.802902,2.445903,2.863952,2.024624,1.822221,6.415880,...,Dovehouse Street,81.0,whole blood,control,NaN,34.0,NaN,0.0,female,0
3,GSM2458695,3.276483,1.755642,3.476344,5.139484,2.124766,2.294782,1.848980,1.900368,5.866003,...,Dovehouse Street,NaN,whole blood,Idiopathic Pulmonary Fibrosis,NaN,NaN,NaN,NaN,NaN,1
4,GSM2458715,3.333986,1.775957,3.361655,8.079835,2.245573,2.354896,1.855805,1.828786,6.120997,...,Dovehouse Street,NaN,whole blood,Idiopathic Pulmonary Fibrosis,NaN,NaN,NaN,NaN,NaN,1
5,GSM2458580,3.477777,2.035227,3.844806,6.712235,2.354887,2.607618,1.754886,2.393481,6.195866,...,Dovehouse Street,70.0,whole blood,control,NaN,34.0,NaN,0.0,male,0
6,GSM2458650,3.558825,1.882950,3.481220,5.294359,2.392698,2.414541,2.019003,2.065364,5.519471,...,Dovehouse Street,NaN,whole blood,Idiopathic Pulmonary Fibrosis,NaN,NaN,NaN,NaN,NaN,1
7,GSM2458595,3.557683,1.865339,3.442669,6.606274,2.273096,2.340224,1.855640,1.920693,5.686922,...,Dovehouse Street,NaN,whole blood,Idiopathic Pulmonary Fibrosis,NaN,NaN,NaN,NaN,NaN,1
8,GSM2458722,3.469403,1.906257,3.650277,6.667523,2.206977,2.623612,2.052782,1.721628,5.744727,...,Dovehouse Street,57.0,whole blood,Idiopathic Pulmonary Fibrosis,82.9,25.0,61.3,0.0,male,1
9,GSM2458678,3.357886,1.877375,3.341571,6.537037,2.470516,2.479536,2.128910,2.004325,6.164376,...,Dovehouse Street,NaN,whole blood,Idiopathic Pulmonary Fibrosis,NaN,NaN,NaN,NaN,NaN,1
